In [4]:
import re
import pandas as pd


In [10]:
df = pd.read_csv('Housing_v2.csv')


In [11]:

# 2. Split the Location field by commas into a list of tokens
df['tokens'] = df['Location'].str.split(r'\s*,\s*')


In [22]:
def extract_postal(tokens):
    # guard: must be a non-empty list
    if not isinstance(tokens, list) or len(tokens) == 0:
        return None
    last = tokens[-1]
    return last if isinstance(last, str) and re.fullmatch(r'\d{4}', last) else None

df['postal_code'] = df['tokens'].apply(extract_postal)

In [30]:
def peel(tokens):
    keys = ['street_or_po','barangay','city_municipality','province']
    # initialize all to None
    out = dict.fromkeys(keys, None)

    if not isinstance(tokens, list) or len(tokens) == 0:
        return pd.Series(out)

    # strip postal code if present
    if isinstance(tokens[-1], str) and re.fullmatch(r'\d{4}', tokens[-1]):
        tokens = tokens[:-1]

    # assign from back with guards
    if len(tokens) >= 1:
        out['province'] = tokens[-1]
    if len(tokens) >= 2:
        out['city_municipality'] = tokens[-2]
    if len(tokens) >= 3:
        out['barangay'] = tokens[-3]
    if len(tokens) > 3:
        out['street_or_po'] = ", ".join(tokens[:-3])

    return pd.Series(out)


In [31]:
parsed = df['tokens'].apply(peel)    # this is a DataFrame with the 4 cols
df = pd.concat([df, parsed], axis=1)

In [32]:
# 6. Drop helper column
df = df.drop(columns=['tokens'])



In [33]:
# 7. Inspect
print(df[['Location','street_or_po','barangay','city_municipality','province','postal_code']])



                                               Location street_or_po barangay  \
0                                 Santo Domingo, Cainta         None     None   
1                              San Vicente, Santa Maria         None     None   
2            Camella Tarlac Access Rd. Maliwalo, Tarlac         None     None   
3                                         Gabon, Abucay         None     None   
4                       Zambowood Rd. Boalan, Zamboanga         None     None   
...                                                 ...          ...      ...   
1495                                   Asisan, Tagaytay         None     None   
1496                                      Pooc, Talisay         None     None   
1497                            Western Bicutan, Taguig         None     None   
1498                                           Antipolo         None     None   
1499  Lot 69 Block 2 Pueblo El Grande Subdivision Ph...         None     None   

                           

In [ ]:
# 8. Save back to Excel if you like
df.to_excel('houses_parsed.xlsx', index=False)